# Postgres Permissions

### Introduction

In the last lesson, we saw how to get started with creating roles, and enabling password protection.  In this lesson, we'll learn a structure for managing read and write permissions with our postgres database.  Let's get started.

### Managing Users

Our general structure for managing users will be the following: 

We'll have three users.  The master user, postgres, and a role for read access and a role for both read and write access.

*  Master user, here postgres.  Will only be used for administrative operations like creating other users, and creating databases.  
* Readonly role: perform readonly access on a given database
* Readwrite role:  Has both read and write access on a given database.

This structure aligns with roles within a company.  For example, as data analysts likely will not need to alter data, but only select data from a database, they can be given the readonly role.  And as web developers may need to both add and destroy records they would have readwrite access.  Finally, someone that manages the database, like devops or a data engineer, would have access to the postgres user for creating new databases or users.

Ok, let's get started.

### Creating a Read Only User

We can begin by logging into postgres as the postgres user, and then creating a readonly user.

To do so, we execute the following commands.

* `CREATE USER readonly WITH PASSWORD 'readonly';`
* `ALTER ROLE readonly WITH LOGIN;`

The first command allowed us to login as that user.  Now, while still the postgres user, we can grant rights to that new `readonly` user.

To do so, we should first determine the database that we want to grant rights to.  In this case, let's grant rights to the blogger database.  

Ok so to do this, the first step is to connect to the blogger database as a postgres user. Then our postgres user will grant read access to the readonly user.

> Why do we need to be the postgres user to do thist?  Remember, we want our master user, here postgres, to be in charge of granting and removing access rights.  Ok, let's connect as the postgres user.

* `\c blogger postgres`



And now that we are connected to the database as the postgres user, we can grant SELECT access to all tables in the database with the following:

```SQL 
GRANT SELECT ON ALL TABLES IN SCHEMA public TO readonly;
```

> Or if we just want to give read only access on certain tables, we can do so with the following:
> ```SQL
> GRANT SELECT ON TABLE mytable1, mytable2 TO readonly;
> ```

We can check that this occurred with something like the following:


```SQL 
SELECT grantee, privilege_type 
FROM information_schema.role_table_grants 
WHERE table_name='posts'
```

> <img src="./show-privileges-on-table.png" width="50%">

So here, we can see that our postgres user has all access to the tables in the database, while the readonly user only has SELECT access.

Then to ensure that the readonly user has access to tables created in the future, we also issue the following command:

```SQL
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT ON TABLES TO readonly;
```

### Granting read and write access

So next up is create a readwrite user, and then grant that user both SELECT and INSERT rights to the database.  Once again, to accomplish this we should first make sure we are connected to the blogger database as the postgres user.

Then we can create our readwrite user:

* CREATE USER readonly WITH PASSWORD 'readonly';
* ALTER ROLE readonly WITH LOGIN;

And grant the appropriate rights to our blogger database: 

```SQL
GRANT CONNECT ON DATABASE practice TO readwrite;

GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA public TO readwrite;
```

So now the readwrite user can perform any operation on the tables.

```SQL
ALTER DEFAULT PRIVILEGES IN SCHEMA public GRANT SELECT, INSERT, UPDATE, DELETE ON TABLES TO readwrite;
```

### Inheriting Group rights

So far we've created two users, readonly and readwrite, who have the corresponding access to our blogger database.  Now let's say we just hire a new developer.  One option is to tell him the `readwrite` password so he can log in as that user.  But if we wish to have a log of his actions on the database, or perhaps be able to revoke those privileges at a certain time, it may be better to create a separate user just for that developer.

```SQL
CREATE USER jeffk WITH PASSWORD 'jeffk';
ALTER ROLE jeffk WITH LOGIN;
```

Then if we want him to have readwrite access to the blogger database, we can make him a member of the readwrite group with the following: 

```SQL
GRANT readwrite TO jeffk;
```

So now he can insert or select from the tables in the blogger database, as he inherits the rights of the readwrite user.

```SQL
SELECT * FROM posts;
```

Then if we want, we can always revoke these rights from the user.

```SQL
REVOKE readwrite FROM jeffk;
```

```SQL
blogger=> SELECT * FROM posts;
ERROR:  permission denied for table posts
```

### Summary

In this lesson, we learned about managing users in postgres.  In doing so we learned about the following commands:

* `CREATE USER readonly WITH PASSWORD 'readonly';`
* `ALTER ROLE readonly WITH LOGIN;`: grant login capabilities
* `GRANT SELECT ON ALL TABLES IN SCHEMA public TO readonly;`: grant select access on all tables

* `GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA public TO readwrite;`: grant read write access
* `GRANT readwrite TO jeffk;`: Grant readwrite permissions to other user
* `REVOKE readwrite FROM jeffk;` Revoke readwrite permissions from user

### Resources

[AWS Blog Postgres Users](https://aws.amazon.com/blogs/database/managing-postgresql-users-and-roles/)

[User Permissions](https://flaviocopes.com/postgres-user-permissions/)

[Aiven Postgres Intro](https://aiven.io/blog/an-introduction-to-postgresql)